In [1]:
cwd = "/home/zzz0054/chen_data"
reads = ["C0RPF1aux_S3_R1_001", "C0RPF2aux_S4_R1_001", "C0RPF3aux_S9_R1_001"]
subF = "Alonso"

In [2]:
!mkdir -p {cwd}/trimmed_out/
!mkdir -p {cwd}/before_trim/
!mkdir -p {cwd}/quality_filtered
!mkdir -p {cwd}/after_trim/
!mkdir -p  {cwd}/aligned

In [3]:
for read in reads:
    !cutadapt -m 15 -M 35 --match-read-wildcards -a AGATCGGAAGAGCACACGTCTGAACTCCAGTCA -o {cwd}/trimmed_out/{read}.trimmed.fastq {cwd}/{subF}/{read}.fastq > {cwd}/trimmed_out/{read}_trimmed.log
    
    !fastqc {cwd}/{subF}/{read}.fastq -o {cwd}/before_trim/ -a {cwd}/adapter.txt
    
    !fastq_quality_filter -Q33 -v -q 25 -p 75 -i {cwd}/trimmed_out/{read}.trimmed.fastq -o {cwd}/quality_filtered/{read}.trimmed.Qfilter.fastq > {cwd}/quality_filtered/{read}.Qfilter.log
    
    !fastqc {cwd}/quality_filtered/{read}.trimmed.Qfilter.fastq -o {cwd}/after_trim/ -a {cwd}/adapter.txt
    
    !bowtie -n 2 -y -a --norc --best --strata -S -p 16 -l 25 --un={cwd}/aligned/noncontam_{read}.fastq {cwd}/bowtie_index/bowtie_index -q {cwd}/quality_filtered/{read}.trimmed.Qfilter.fastq {cwd}/aligned/{read}.alin
    
    !STAR --runThreadN 8 --outFilterType Normal --outWigType wiggle --outWigStrand Stranded --outWigNorm RPM --alignEndsType EndToEnd --outFilterMismatchNmax 1 --outFilterMultimapNmax 1 --genomeDir {cwd}/STAR_genome_index/ --readFilesIn {cwd}/aligned/noncontam_{read}.fastq --outFileNamePrefix  {cwd}/star_out/{read}/ --outSAMtype BAM SortedByCoordinate --quantMode TranscriptomeSAM GeneCounts --outSAMattributes All
    
    !samtools sort -T {cwd}/star_out/{read}/Aligned.toTranscriptome.out.sorted -o {cwd}/star_out/{read}/Aligned.toTranscriptome.out.sorted.bam {cwd}/star_out/{read}/Aligned.toTranscriptome.out.bam
    !samtools index {cwd}/star_out/{read}/Aligned.toTranscriptome.out.sorted.bam ## mapped to transcriptome
    !samtools index {cwd}/star_out/{read}/Aligned.sortedByCoord.out.bam ## mapped to genome
    
    !mkdir {cwd}/metaplots/{read}
    !metaplots -a {cwd}/ref/annot -r {cwd}/star_out/{read}/Aligned.toTranscriptome.out.sorted.bam -o {cwd}/metaplots/{read}/

[        8=--] 00:02:36    16,500,000 reads @   9.5 µs/read;   6.29 M reads/minute